In [1]:
from flask import Flask, request, render_template_string
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
import os

In [2]:
# HTML template for the web page
HTML = '''
<!DOCTYPE html>
<html>
<body>
<h1>Dog Breed Prediction</h1>
<form action="/" method="post" enctype="multipart/form-data">
  <input type="file" name="file" required>
  <input type="submit" value="Upload Image">
</form>
{{ prediction_text }}
</body>
</html>
'''


In [ ]:
# Initialize the Flask application
app = Flask(__name__)

# Load the pre-trained Keras model (make sure the model path is correct)
model = load_model('model.h5')

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Check if the post request has the file part
        if 'file' not in request.files:
            return render_template_string(HTML, prediction_text='No file part')
        file = request.files['file']
        if file.filename == '':
            return render_template_string(HTML, prediction_text='No selected file')

        if file:
            # Save the file to the uploads folder
            basepath = os.path.dirname(__file__)
            file_path = os.path.join(basepath, 'uploads', file.filename)
            file.save(file_path)

            # Make prediction
            img = image.load_img(file_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)  # Create a batch

            predictions = model.predict(img_array)
            predicted_class = np.argmax(predictions, axis=1)  # Assuming class indices are breed names

            # Map the predictions to actual breed names
            breeds = ['breed1', 'breed2', 'breed3', ..., 'breed120']  # Update with actual breed names
            predicted_breed = breeds[predicted_class[0]]

            return render_template_string(HTML, prediction_text=f'Predicted Breed: {predicted_breed}')

    return render_template_string(HTML, prediction_text='')

if __name__ == '__main__':
    app.run(debug=True, port=5000)
